# Scraper para obtener datos de servidores públicos

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

## Conexión al navegador

In [38]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://nominatransparente.rhnet.gob.mx/nomina-APF'
driver.get(start_url)
driver.execute_script("document.body.style.zoom='50%'") 

    

## Lectura de archivo csv para obtener nombres

In [40]:
df = pd.read_csv('ejemplo.csv', keep_default_na=False) 
df['NombreCompleto'] = df.apply(lambda row: f"{row['Nombre']} {row['Paterno']} {row['Materno']}", axis=1)

df

,Nombre,Paterno,Materno,NombreCompleto
0,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
1,Luis,Cruz,Jaramillo,Luis Cruz Jaramillo
2,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
3,José Alberto,Montán,López,José Alberto Montán López
4,Andres Manuel,Lopez,Obrador,Andres Manuel Lopez Obrador
5,Jorge Luis,Lopez,,Jorge Luis Lopez
6,Juan de Dios,Olivares,Jimenez,Juan de Dios Olivares Jimenez
7,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
8,José Alberto,Montán,López,José Alberto Montán López


In [41]:
data = [ ]

for i in df['NombreCompleto']:
    print(i)
    #WebDriverWait(driver, 10)
    nombre = driver.find_element(By.XPATH, '//*[@id="apfName"]')
    nombre.send_keys(i)
    #WebDriverWait(driver, 10)
    buscar = driver.find_element(By.XPATH, '/html/body/app-root/app-apf/div[2]/div[1]/div[3]/div/div[3]/a')
    #WebDriverWait(driver, 10)
    buscar.click()
    
    time.sleep(5)
    # Obtener identificadores de ventanas
    ventanas = driver.window_handles
    
    # Cambiar al controlador de la ventana emergente
    
    try:
    # Buscar y hacer clic en el botón dentro de la ventana emergente
        driver.switch_to.window(ventanas[-1])
        ok_button = driver.find_element(By.CSS_SELECTOR, 'button.swal2-confirm.swal2-styled')
        ok_button.click()
        print('Se encontró el botón')
    except NoSuchElementException:
        html_table = driver.page_source
    
        # Crear un objeto BeatifulSoup para analizar el html
        soup = BeautifulSoup(html_table, 'html.parser')
        
        # Encontrar todas las finlas de la tabla
        rows = soup.find_all('tr')
        
        for row in rows:
            cells = row.find_all('td')
            # Extraer el texto de cada celda y agregarlo a la lista de datos 
            row_data = [cell.get_text(strip=True) for cell in cells]
            row_data = [cell.split(":")[1].strip() for cell in row_data if ":" in cell]
            data.append(row_data)
        nombre.clear()
        
        pass

    
        
df = pd.DataFrame(data)
    
# Asignar nombres a las columnas
df.columns = ['Número', 'Nombre', 'Institución', 'Puesto', 'Sueldo_bruto_mensual', 'Sueldo_neto_estimado' ]
df.dropna(inplace=True)
df

    
   
    
    

    # Ingresar nombre
    

Bryan Antonio Polito Palma


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (1410, 1031)
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x011AA813+48355]
	(No symbol) [0x0113C4B1]
	(No symbol) [0x01045358]
	(No symbol) [0x01075062]
	(No symbol) [0x01073B98]
	(No symbol) [0x0107239B]
	(No symbol) [0x0107179E]
	(No symbol) [0x01069F5C]
	(No symbol) [0x0108A73C]
	(No symbol) [0x01069A36]
	(No symbol) [0x0108AA94]
	(No symbol) [0x0109C922]
	(No symbol) [0x0108A536]
	(No symbol) [0x010682DC]
	(No symbol) [0x010693DD]
	GetHandleVerifier [0x0140AABD+2539405]
	GetHandleVerifier [0x0144A78F+2800735]
	GetHandleVerifier [0x0144456C+2775612]
	GetHandleVerifier [0x012351E0+616112]
	(No symbol) [0x01145F8C]
	(No symbol) [0x01142328]
	(No symbol) [0x0114240B]
	(No symbol) [0x01134FF7]
	BaseThreadInitThunk [0x773805C9+25]
	RtlGetAppContainerNamedObjectPath [0x77D977AD+237]
	RtlGetAppContainerNamedObjectPath [0x77D9777D+189]
